In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plot

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# CIFAR_10 is a set of 60k images 32x32 pixels on 3 channels
# Sets the number of color channels in the input images.
# 3 is for RGB
IMG_CHANNELS = 3

# sets width to 32px
IMG_ROWS = 32

# set the height to 32px
IMG_COLS = 32

#constant

# is the number of samples that will be propagated
# through the neural network at once.
BATCH_SIZE = 128
# number of rounds the data will be passed 
# through the neural network
NB_EPOCH = 20
# Classes or categories the input data can be
# classified into.
NB_CLASSES = 10
# Controls the amount of information printed
# during the training process.
VERBOSE = 1
# This line sets a fraction of the training data
# that will be used for validation during training.
VALIDATION_SPLIT = 0.2
# the optimizer algorithm is RMSprop
OPTIM = RMSprop()


#load dataset

# mapping the elements of the image data to x values with labels
# and y values with labels
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert the class labels from integers to one-hot
# encoded vectors.
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

## converts the results to float32 integers (floating-point numbers)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# does some weird division by 255 on the x values
# apparently its common practice
X_train /= 255
X_test /= 255

# network
# initializes the model
model = Sequential()


# layer one
# creates the convulation layer for layer one
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))

# add the activation function (ReLU) to this convolutional layer
model.add(Activation('relu'))

# adding a 2nd convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))

# adding a 2nd activation function for this onvolutional layer
model.add(Activation('relu'))

# adding a max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# adding a dropout to the layer
model.add(Dropout(0.25))

# layer two
# creates the convulation layer for layer two - not sure why it needs to be 64 over 32 this time around?
model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))

# add the activation function (ReLU) to this convolutional layer
model.add(Activation('relu'))

# adding a 2nd convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3)))

# adding a 2nd activation function for this onvolutional layer
model.add(Activation('relu'))

# adding a max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# adding a dropout to the layer
model.add(Dropout(0.25))

# flattens the multi-dimensional output
model.add(Flatten())


# combines and processes the features learned by the convolutional and pooling layers
model.add(Dense(512))

# add an activation function for the fully connected layer
model.add(Activation('relu'))

# add a dropout to the fully connected layer
model.add(Dropout(0.5))

# adds the classes/categories to the final output
model.add(Dense(NB_CLASSES))

# adds tge softmax activation function to the output
model.add(Activation('softmax'))



<img src="./1.png" alt="1" width="600" />

In [ ]:
# results
model.summary()

<img src="./2.png" alt="2" width="600" />
<img src="./3.png" alt="3" width="600" />

In [ ]:
# trains the data
# and adds all the parameters to the neural network
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)

print("Test score:", score[0])
print('Test accuracy:', score[1])

<img src="./4.png" alt="4" width="600" />

## How can training machines on images be harmful? What are the negative impacts of the technology on our society

Training machines on images can be harmful because they inherently introduce bias by design. Beyond training the DCNN on biased data, the pooling layer, specifically the max pooling layer, starts a process that is inherently biased because it mathematically prioritizes some features of the element over others and generalizes the data. This is helpful from a computational perspective, but it augments the data at a fundamental level. One negative impact of this process is that it can make the data insensitive to small shifts, which can result in biased outcomes. 'In Max Pooling, small changes in the non-maximal values will be ignored. This insensitivity to small changes can be problematic and can bias the results (Arham, 2023).' Therefore, by introducing some level of favoritism in the data at the core level of the DCNN, the output will always exhibit some degree of bias, regardless of the input.

If by design, the DCNN introduces bias into the training data, applying that to images can ultimately have a negative impact on our society. Since it's designed to emphasize certain features over others, if we don't provide a diverse set of training data, it'll compound common features found in all the training images. For example, Joy Boulamwini, a computer scientist who studied machine learning at MIT, discovered there was "large gender and racial bias in AI systems sold by tech giants like IBM, Microsoft, and Amazon (Boulamwini, 2019)." These AI systems were better at recognizing lighter-skinned men than dark-skinned women, with "1% error for lighter-skinned men. For darker-skinned women, the errors soared to 35%. AI systems from leading companies have failed to correctly classify the faces of Oprah Winfrey, Michelle Obama, and Serena Williams (Boulamwini, 2019)."  With such a skewed outputs, these systems should be examined more thoroughly first by taking a look closely at the type of image data behind fed to the sytem and second looking at how the pooling layer augments the data an introduces unexpected bias.


## For example, could such an algorithm eventually be used to distinguish people’s faces? If so, what are the ethical and privacy implications?

When passing image data to a neural network, it's important to consider the potential negative effects, particularly from an ethical standpoint. For instance, feeding images of people into a Deep Convolutional Neural Network (DCNN) prompts the algorithm to analyze and interpret these images. If the developers of the DCNN persistently refine the system to recreate images with high accuracy and likeness based on the input data, this can lead to significant ethical concerns. One problem based on this fact, which is also is becoming more prevalent in our society: deepfakes, which are "kind of synthetic media where a person in an image or video is swapped with another person’s likeness (Dhiman, 2023)." Even though the code we ran seemed pretty interesting at the time, it could easily be refactored to generate deepfake photos based on the training data provided, raising further a very important ethical concerns.

There isn't much argument against how deepfakes are unethical and invade personal privacy. A recent example of the impact how damaging deepfakes can be is what happened to Taylor Swift. Taylor Swift was a victim of deepfake with her face imposed over pornographic images and shared on social media (Associated Press, 2024). As of right now the source of these images is unknown, but whoever is behind it they are clearly wrong morally and legally.  Since the incident according to the BBC, "US politicians have called for new laws to criminalise the creation of deepfake(Rahman-Jones, 2024)" based on this incident. This case highlights the urgent need for legislation that specifically targets deepfake technology to protect individuals' rights and privacy.

**References:**

Buolamwini, J. (2019, February 7). Generative AI: Risks and limits on journalism and society. Media Diversity Institute. Retrieved from [https://www.media-diversity.org/generative-ai-risks-and-limits-on-journalism-and-society/](https://www.media-diversity.org/generative-ai-risks-and-limits-on-journalism-and-society/)

Arham, M. (2023, September 28). Diving into the pool: Unraveling the magic of CNN pooling layers. KDnuggets. Retrieved from [https://www.kdnuggets.com/diving-into-the-pool-unraveling-the-magic-of-cnn-pooling-layers](https://www.kdnuggets.com/diving-into-the-pool-unraveling-the-magic-of-cnn-pooling-layers)

Dhiman, B. (2023, December 7). Exploding AI-generated deepfakes and misinformation: A threat to global concern in the 21st century. ResearchGate. [https://www.researchgate.net/publication/376329868_Exploding_AI-Generated_Deepfakes_and_Misinformation_A_Threat_to_Global_Concern_in_the_21st_Century](https://www.researchgate.net/publication/376329868_Exploding_AI-Generated_Deepfakes_and_Misinformation_A_Threat_to_Global_Concern_in_the_21st_Century)

Associated Press. (2024, January 26). Fans fight back as fake explicit images of Taylor Swift spread on social media. MSN. [https://www.msn.com/en-us/news/technology/fans-fight-back-as-fake-explicit-images-of-taylor-swift-spread-on-social-media/ar-BB1hjPDV](https://www.msn.com/en-us/news/technology/fans-fight-back-as-fake-explicit-images-of-taylor-swift-spread-on-social-media/ar-BB1hjPDV)

Rahman-Jones, I. (2024, January 27). Taylor Swift deepfakes spark calls in Congress for new legislation. BBC News. [https://www.bbc.com/news/technology-68110476?embed=true](https://www.bbc.com/news/technology-68110476?embed=true)